# 4. Wh- Question

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pseudo-Lab/Tutorial-Book/blob/master/book/chapters/NLP/Ch4-Wh-Question.ipynb)

3장에서는 각 지문에 대한 True/False 문제를 생성해보았습니다. 이번 4장에서는 T5모델과 BERT모델로 Wh- 문제를 생성해보겠습니다. Cambridge Dictionary에 의하면 [Wh- 문제](https://dictionary.cambridge.org/ko/%EC%82%AC%EC%A0%84/%EC%98%81%EC%96%B4/wh-question)는 what, when, where, who, whom, which, whose, why, how로 시작하는 문제라고 정의되어 있습니다. 이번장에서는 이러한 Wh- 문제를 다지선다(Multi-Choice)로 생성해 볼 것입니다.

4.1절에서는 모델의 입력값으로 사용할 에세이 데이터를 불러오고 4.2절에서는 정답 단어를 추출하는 함수를 만들어 보겠습니다. 4.3절에서는 Wh- 문제를 생성하는 클래스, 4.4절에서는 문제를 평가하는 클래스, 그리고 4.5절에서는 오답을 생성하는 클래스를 정의하도록 하겠습니다. 마지막으로 4.6절에서는 정의한 클래스들을 사용하여 실제 에세이 지문에서 Wh- 문제를 생성해보도록 하겠습니다.

## 4.1 데이터셋 다운로드

문제 생성에 앞서 2장에서 저장한 데이터셋을 불러오도록 하겠습니다. 이 데이터셋은 전처리를 통해 올바른 표현으로 수정하여 저장해둔 데이터셋입니다. 2장에서 직접 저장한 파일을 불러오거나, 혹은 가짜연구소 깃허브에 저장된 파일을 불러와 사용할 수 있습니다. `CoNLL+BEA_corrected_essays.pkl` 파일 안에는 총 170개의 지문이 존재하며, 이 데이터는 Wh- 문제를 만들 지문으로 사용됩니다.

In [1]:
!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data NLP-QG

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 9), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.
CoNLL+BEA_corrected_essays.pkl is done!


In [2]:
import pickle
file_name = "CoNLL+BEA_corrected_essays.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [3]:
len(data)

170

첫 번째 지문을 예시로 출력해보겠습니다.

In [4]:
print(data[0])

Keeping the Secret of Genetic Testing What is genetic risk? Genetic risk refers  to your chance of inheriting a disorder or disease. People get certain diseases because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However, for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagnosed with certain genetic diseases, are we suppose to disclose this result to our relatives? My answer is no. On one hand, we do not want this potential danger havingfrightening effects in our families' later lives. Whe

함수를 정의하기 전에 4장에서 사용할 패키지들을 import 해보겠습니다. 각 환경에 따라 패키지가 없을 수 있으니 확인하시고 설치하시기 바랍니다. (Colab 환경에서는 `benepar` 패키지가 없어 아래 코드를 통해 설치해줍니다.)

`benepar`와 `nltk`는 텍스트 파싱과 토크나이징에 사용할 수 있는 패키지입니다. 또한 `pandas`는 기본적인 데이터 프레임을 위한 패키지이고, `numpy`는 수치연산에 사용하는 패키지입니다. 그리고 `torch`는 모델 구축 프레임워크이고, 다양한 토큰화 기법과 모델들이 내장되어 있는 `transformers` 패키지도 있습니다.

In [5]:
!pip install -q benepar

     |████████████████████████████████| 3.3 MB 7.5 MB/s 
     |████████████████████████████████| 2.8 MB 36.4 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 34.3 MB/s 
     |████████████████████████████████| 636 kB 46.1 MB/s 


In [6]:
import benepar
benepar.download('benepar_en3')
benepar_parser = benepar.Parser("benepar_en3")

import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize

import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    pipeline
)

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


본격적인 모델링에 앞서 실험 환경에서 GPU가 사용 가능한지 확인할 수 있습니다. `cuda`가 출력되면 GPU를 사용할 수 있는 것입니다. 단, GPU가 없다면 CPU를 사용해도 가능합니다.

In [7]:
print('cuda' if torch.cuda.is_available() else 'cpu')

cuda


## 4.2 정답 단어 추출

Wh- 문제를 생성하기 위한 첫 번째 단계는 지문에서 정답으로 사용할 단어를 추출하는 것입니다. 각 지문에서 `benepar_parser tree`를 통해 개체명을 인식하고 명사구에 해당하는 단어들만 추출하여 리스트로 저장합니다.

In [8]:
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final

def get_NP(doc):
    answers = []
    trees = benepar_parser.parse_sents(sent_tokenize(doc))
    for sent_idx, tree in enumerate(trees):
        subtrees = tree.subtrees()
        for subtree in subtrees:
            if subtree.label() == "NP":
                answers.append(get_flattened(subtree))
    return answers    

## 4.3 문제 생성 클래스 정의

이번에는 Wh- 문제를 생성하는 클래스를 정의해보겠습니다. 위에서 추출한 정답 단어들과 지문을 T5기반의 Seq2SeqLM모델에 넣어 문제를 생성하게 됩니다. 여기서 사용된 토크나이저와 모델은 [huggingface](https://huggingface.co/models)가 제공하는 사전학습모델(API)을 가져와 사용하였습니다. 이 때, AutoModel API는 학습 가중치에 넣고자 하는 모델을 자동으로 찾아서 생성해주는 API입니다. 예를 들어 T5모델을 경로에 넣어주면 자동으로 T5구조를 생성해 그 안에 학습 가중치를 넣어주고, BERT모델을 경로에 넣어주면 BERT구조를 생성해 학습 가중치를 넣어주게 됩니다. 이번 QuestionGenerator 클래스에서는 [iarfmoose/t5-base-question-generator](https://huggingface.co/iarfmoose/t5-base-question-generator)를 불러왔습니다.

In [9]:
class QuestionGenerator():
    def __init__(self):
        QG_PRETRAINED = "iarfmoose/t5-base-question-generator"

        self.ANSWER_TOKEN = "<answer>"
        self.CONTEXT_TOKEN = "<context>"
        self.SEQ_LENGTH = 512
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.qg_tokenizer = AutoTokenizer.from_pretrained(QG_PRETRAINED, use_fast=False)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(QG_PRETRAINED)
        self.qg_model.to(self.device)
        self.qg_model.eval()

    def generate_question(self, answers):
        questions = []

        for ans in answers: 
            qg_input = "{} {} {} {}".format(self.ANSWER_TOKEN, ans, self.CONTEXT_TOKEN, passage)
            
            encoded_input = self.qg_tokenizer(qg_input, padding='max_length', max_length=self.SEQ_LENGTH, truncation=True, return_tensors="pt").to(self.device)
            with torch.no_grad():
                output = self.qg_model.generate(input_ids=encoded_input["input_ids"])
            question = self.qg_tokenizer.decode(output[0], skip_special_tokens=True)
            questions.append(question)
        return questions        

In [10]:
q_generator = QuestionGenerator()

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/121 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

## 4.4 문제 평가 클래스 정의

다음으로 생성된 문제와 정답쌍을 평가하는 클래스를 생성해보겠습니다. 문제와 정답간의 의미적 관련성을 Sequence Classification모델을 통해 점수매기게 됩니다. QAEvaluator 클래스에서는 [iarfmoose/bert-base-cased-qa-evaluator](https://huggingface.co/iarfmoose/bert-base-cased-qa-evaluator)를 사용하였습니다.

In [11]:
class QAEvaluator():
    def __init__(self):
        QAE_PRETRAINED = "iarfmoose/bert-base-cased-qa-evaluator"
        self.SEQ_LENGTH = 512

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.qae_tokenizer = AutoTokenizer.from_pretrained(QAE_PRETRAINED)
        self.qae_model = AutoModelForSequenceClassification.from_pretrained(QAE_PRETRAINED)
        self.qae_model.to(self.device)

    def encode_qa_pairs(self, questions, answers):
        encoded_pairs = []
        for i in range(len(questions)):
            encoded_qa = self.qae_tokenizer(text=questions[i], text_pair=answers[i], padding="max_length", max_length=self.SEQ_LENGTH, truncation=True, return_tensors="pt")
            encoded_pairs.append(encoded_qa.to(self.device))
        return encoded_pairs

    def get_scores(self, encoded_qa_pairs):
        scores = {}
        self.qae_model.eval()
        with torch.no_grad():
            for i in range(len(encoded_qa_pairs)):
                scores[i] = self.qae_model(**encoded_qa_pairs[i])[0][0][1]
        return [k for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)]  

In [12]:
qa_evaluator = QAEvaluator()

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

## 4.5 오답 생성 클래스 정의

마지막으로 오답을 생성하는 클래스를 정의해보겠습니다. 4.3절에서 추출한 정답 단어를 제외하고 4개의 보기 중 3개의 오답이 필요합니다. 이는 정답 단어에서 명사에 해당하는 첫 단어를 추출해 마스킹하고, BERT모델을 통해 그 부분을 다른 단어로 생성하게 됩니다. DistractorGenerator 클래스에서는 [bert-base-cased](https://huggingface.co/bert-base-cased)를 사용하였습니다.

In [13]:
class DistractorGenerator():
    def __init__(self):
        self.unmasker = pipeline('fill-mask', model='bert-base-cased')  

    def generate_distractor(self, text, candidate, answers, NNs: list):
        distractor = []
        divided = word_tokenize(text)
        substitute_word = NNs[0]

        mask_index = divided.index(substitute_word)
        divided.pop(mask_index)

        divided.insert(mask_index, '[MASK]')
        text = ' '.join(divided)
        unmasked_result = self.unmasker(text, top_k=10)[candidate]

        text = unmasked_result["sequence"]

        answers = answers.split(' ')
        answer_index = answers.index(substitute_word)
        answers.pop(answer_index)
        answers.insert(answer_index, unmasked_result["token_str"])
        return " ".join(answers)

In [14]:
def get_NN(distractor):
      NNs = []
      tree = benepar_parser.parse(distractor)
      subtrees = tree.subtrees()
      for subtree in subtrees:
          if subtree.label() in ["NN", "NNP", "NNS", "VB"]: #VB for edge case
              NNs.extend(subtree.leaves())       
      return NNs

In [15]:
d_generator = DistractorGenerator()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

## 4.6 Wh- 문제 생성

지금까지 정의한 클래스를 사용하여 지문에 대한 Wh- 문제를 생성해보겠습니다. 먼저 생성한 문제와 보기들을 저장할 데이터 프레임을 생성하고, 사용할 지문 20개를 선정합니다.(20개의 번호는 임의로 선정하였습니다.)

In [16]:
df_WHQuestions = pd.DataFrame({'id': np.zeros(20),
                               'passage': np.zeros(20),
                               'question': np.zeros(20),
                               'distractor_1': np.zeros(20),
                               'distractor_2': np.zeros(20),
                               'distractor_3': np.zeros(20),
                               'distractor_4': np.zeros(20)})

passage_id_list = [163, 28, 62, 57, 35, 26, 22, 151, 108, 55, 59, 129, 167, 143, 50, 161, 107, 56, 114, 71]

정의한 클래스들을 순서대로 진행합니다. 중간에 정답이 4단어를 넘지 않도록 제한하는 과정과 오답을 생성하기 전에 정답이 있는 문장을 찾아주는 과정이 포함되어 있습니다. 그리고 최종 생성한 문제, 정답, 오답은 데이터 프레임에 저장합니다.

In [17]:
df_idx = 0

for passage_id in passage_id_list:

    passage = data[passage_id]

    answers = get_NP(passage)

    questions = q_generator.generate_question(answers)

    encoded_qa_pairs = qa_evaluator.encode_qa_pairs(questions, answers)
    scores = qa_evaluator.get_scores(encoded_qa_pairs)
    
    ## 정답의 단어 개수 len() <= 4 사용한다. 
    for i in range(len(scores)):
        index = scores[i]
        if len(answers[index].split(' ')) > 4:
            continue
        break
    
    sentences = nltk.sent_tokenize(passage)
    for sentence in sentences:
        if answers[index] in sentence:
            target_sentence = sentence

    NNs = get_NN(answers[index])

    distractors = []
    for i in range(3):
        distractors.append(d_generator.generate_distractor(target_sentence, 9-i, answers[index], NNs))
    
    df_WHQuestions.loc[df_idx] = [passage_id, passage, questions[index].split("?")[0] + "?", answers[index]] + distractors
    print(f"finished {passage_id}")
    df_idx += 1

/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


finished 163
finished 28
finished 62
finished 57
finished 35
finished 26
finished 22
finished 151
finished 108
finished 55
finished 59
finished 129
finished 167
finished 143
finished 50
finished 161
finished 107
finished 56
finished 114
finished 71


최종 생성된 결과는 다음과 같습니다.

In [18]:
df_WHQuestions

,id,passage,question,distractor_1,distractor_2,distractor_3,distractor_4
0,163.0,The waters of the culinary seas had been calm ...,What is the best definition of a mother who ca...,A mother who works,A cook who works,A kid who works,A waitress who works
1,28.0,The world is increasingly becoming flat with a...,What are the disadvantages of social network s...,The cyber communication,The cyber network,The cyber environment,The cyber community
2,62.0,The best places for y...,What country is the best place to visit for yo...,a different country,a different village,a different generation,a different town
3,57.0,Puerquitour: A great experience for your mouth...,Where did we go to watch the movie Renoir?,a movie theatre,a new theatre,a big theatre,a pizza theatre
4,35.0,"Nowadays, social media sites are commonly used...",What are the advantages of using social media ...,Substantial costs,Substantial parts,Substantial events,Substantial frames
5,26.0,"Interpersonal skills, like any other skills re...",What are the two most popular social media app...,Skype and Facetime,Word and Facetime,Flash and Facetime,email and Facetime
6,22.0,"Nowadays, with the advancement of technology, ...",What is the reason why a person may not have a...,anymore contact,anymore interactions,anymore dealings,anymore sex
7,151.0,In this century there have been many technolog...,What is the most important aspect of television?,The entertainment aspect,The technical aspect,The broadcasting aspect,The political aspect
8,108.0,"I met a friend about one week ago, and he aske...",What is the feeling that you have now?,an awful feeling,an awful question,an awful think,an awful thinking
9,55.0,"Dear Sir or Madam,\nI am writing to apply for ...",What do you hope to get from this job?,valuable experience,valuable ##s,valuable tips,valuable time


생성된 문제와 보기를 보면 문법적으로는 어느 정도 잘 생성해내는 것 같습니다. 하지만 단순하게 명사를 추출하여 보기를 만들었기 때문에 지문에서 크게 중요하지 않은 질문들도 있고 엉뚱한 보기도 있어보입니다. 정답과 오답을 잘 고른다면 좀 더 유의미한 문제를 생성할 것이라고 기대되어집니다.

지금까지 Huggingface에서 제공하는 사전학습모델들로 Wh- 문제를 생성해보았습니다. 만약 SQuAD와 같은 QA 데이터셋이 있다면, 각자 생성하고 싶은 도메인으로 학습시켜 특화된 질문을 만드는데 활용해보시기 바랍니다.

## 4.7 참고문헌

- [Question_Generator](https://github.com/AMontgomerie/question_generator)
- [huggingface](https://huggingface.co/models)
